In [1]:
from config import api_key
import requests
import json
import pandas as pd

In [2]:
url = "https://api.yelp.com/v3/businesses/search"

In [3]:
header = {"Authorization": f"Bearer {api_key}"}

data = []
for _ in range(0,1000,50):
    params = {
        "location": "austin, tx",
        "term": "food trucks",
        "limit": 50,
        "offset": _
    }
    data += requests.get(url, params=params, headers=header).json()["businesses"]

In [ ]:
#print(json.dumps(data, indent=4, sort_keys=True))

In [4]:
len(data)

874

In [5]:
df = pd.DataFrame(data)
df.head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,schallers-stube-sausage-bar-austin-2,"[{'alias': 'german', 'title': 'German'}, {'ali...","{'latitude': 30.26548, 'longitude': -97.74405}",,4581.044355,YmrTE3k7eIUCrUPi18MeMQ,https://s3-media1.fl.yelpcdn.com/bphoto/hl3TA7...,False,"{'address1': '102 1/2 W 3rd St', 'address2': '...",Schaller's Stube Sausage Bar,,NaN,4.5,21,[],https://www.yelp.com/biz/schallers-stube-sausa...
1,jewhungry-austin,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...","{'latitude': 30.22565, 'longitude': -97.76274}",(512) 934-2397,8847.698457,yV01eojIM0Om-LxJVnusMQ,https://s3-media1.fl.yelpcdn.com/bphoto/2RaGrw...,False,"{'address1': '3808 South Congress Ave', 'addre...",Jewhungry?,+15129342397,$,5.0,6,[],https://www.yelp.com/biz/jewhungry-austin?adju...
2,the-picnic-austin,"[{'alias': 'foodtrucks', 'title': 'Food Trucks'}]","{'latitude': 30.2636533696879, 'longitude': -9...",,4644.448492,vLKgesyq-yEfpzeAFgPorQ,https://s3-media3.fl.yelpcdn.com/bphoto/v2CeSq...,False,"{'address1': '1720 Barton Springs Rd', 'addres...",The Picnic,,$$,4.5,80,[],https://www.yelp.com/biz/the-picnic-austin?adj...
3,dolce-bacio-gelato-austin,"[{'alias': 'desserts', 'title': 'Desserts'}, {...","{'latitude': 30.26315, 'longitude': -97.72591}",(512) 945-4542,5583.117217,6A8smdPYPc1u5pP1ATZQug,https://s3-media1.fl.yelpcdn.com/bphoto/nHn5B4...,False,"{'address1': '1606 E 6th St', 'address2': None...",Dolce Bacio Gelato,+15129454542,NaN,5.0,10,"[delivery, pickup]",https://www.yelp.com/biz/dolce-bacio-gelato-au...
4,pastelaria-são-paulo-austin,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...","{'latitude': 30.24441, 'longitude': -97.75808}",,6759.193629,OorxaXtp3JtnkyBQ27TXjg,https://s3-media1.fl.yelpcdn.com/bphoto/fqG8rn...,False,"{'address1': '603 W Live Oak St', 'address2': ...",Pastelaria São Paulo,,NaN,5.0,20,[],https://www.yelp.com/biz/pastelaria-s%C3%A3o-p...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 874 entries, 0 to 873
Data columns (total 16 columns):
alias            874 non-null object
categories       874 non-null object
coordinates      874 non-null object
display_phone    874 non-null object
distance         874 non-null float64
id               874 non-null object
image_url        874 non-null object
is_closed        874 non-null bool
location         874 non-null object
name             874 non-null object
phone            874 non-null object
price            652 non-null object
rating           874 non-null float64
review_count     874 non-null int64
transactions     874 non-null object
url              874 non-null object
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 103.4+ KB


In [7]:
def get_cat(categories):
    cats = []
    for _ in range(len(categories)):
        cat = categories[_]["alias"]
        cats.append(cat)
    return cats

df["categories"] = df["categories"].map(get_cat)
df["latitude"] = df["coordinates"].map(lambda x: x["latitude"])
df["longitude"] = df["coordinates"].map(lambda x: x["longitude"])
df["address"] = df["location"].map(lambda x: x["address1"])
df["zip_code"] = df["location"].map(lambda x: x["zip_code"])

In [10]:
def is_truck(cats):
    if "foodtrucks" in cats:
        return True
    else:
        return False
df["is_truck"] = df["categories"].map(is_truck)
df = df[df["is_truck"]]

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 634 entries, 0 to 866
Data columns (total 21 columns):
alias            634 non-null object
categories       634 non-null object
coordinates      634 non-null object
display_phone    634 non-null object
distance         634 non-null float64
id               634 non-null object
image_url        634 non-null object
is_closed        634 non-null bool
location         634 non-null object
name             634 non-null object
phone            634 non-null object
price            445 non-null object
rating           634 non-null float64
review_count     634 non-null int64
transactions     634 non-null object
url              634 non-null object
latitude         633 non-null float64
longitude        633 non-null float64
address          626 non-null object
zip_code         634 non-null object
is_truck         634 non-null bool
dtypes: bool(2), float64(4), int64(1), object(14)
memory usage: 100.3+ KB


In [12]:
df = df[["id", "name", "categories", "latitude", "longitude", "address", "zip_code", "price", "rating", "review_count", "url"]]
df.head()

,id,name,categories,latitude,longitude,address,zip_code,price,rating,review_count,url
0,YmrTE3k7eIUCrUPi18MeMQ,Schaller's Stube Sausage Bar,"[german, foodtrucks]",30.265480,-97.744050,102 1/2 W 3rd St,78701,NaN,4.5,21,https://www.yelp.com/biz/schallers-stube-sausa...
1,yV01eojIM0Om-LxJVnusMQ,Jewhungry?,"[foodtrucks, delis, kosher]",30.225650,-97.762740,3808 South Congress Ave,78704,$,5.0,6,https://www.yelp.com/biz/jewhungry-austin?adju...
2,vLKgesyq-yEfpzeAFgPorQ,The Picnic,[foodtrucks],30.263653,-97.763167,1720 Barton Springs Rd,78704,$$,4.5,80,https://www.yelp.com/biz/the-picnic-austin?adj...
3,6A8smdPYPc1u5pP1ATZQug,Dolce Bacio Gelato,"[desserts, gelato, foodtrucks]",30.263150,-97.725910,1606 E 6th St,78702,NaN,5.0,10,https://www.yelp.com/biz/dolce-bacio-gelato-au...
4,OorxaXtp3JtnkyBQ27TXjg,Pastelaria São Paulo,"[foodtrucks, brazilian]",30.244410,-97.758080,603 W Live Oak St,78704,NaN,5.0,20,https://www.yelp.com/biz/pastelaria-s%C3%A3o-p...


In [13]:
df.to_csv("data.csv", index=False)